In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from twitchio.ext.commands import Bot, Context, command
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import re
from random import choice


In [ ]:
def get_game_id():
    
    link = ('https://playmafia.pro/cabinet/get?userId=2427&days=0&offset=0&limit=1')
    responce = requests.get(link).text
    soup = BeautifulSoup(responce, 'lxml')
    block_split = (str(soup).rpartition('['))
    block_split = (block_split[1] + block_split[2])
    block_split = (str(block_split).rpartition(']'))
    result = str(block_split[0] + block_split[1])
    data = json.loads(result)
    return(data[0].get("id"))
    
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get("https://playmafia.pro/cabinet/2427")
#     elem = driver.find_element_by_xpath("//*[@id='app']/div/main/div/div/ul/li[2]")

#     elem.click()
#     pagesauce = driver.page_source
#     bs = BeautifulSoup(driver.page_source, "html.parser")

#     aboba = bs.body.select("#app > div > main > div > div > div:nth-child(5) > section > div.history-content.list-wrapper > div.VueTables.VueTables--server > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(6)")
#     split_one = re.split(r">", str(aboba[0]))
#     split_two = re.split(r"<", split_one[1])
#     driver.close()
#     return(split_two[0])


In [ ]:
def get_players(game_id: str) -> str:
    
    try:
        html_body = requests.get(f"https://playmafia.pro/game-statistics/{game_id}").text
        bs = BeautifulSoup(html_body, "html.parser")
        gamestats = json.loads(bs.gamestats[":game-data"])
        players = gamestats["players"]

        mafia = []
        sheriff = None
        winner = ""

        for player in players:
            if player["role"]["type"] in ["mafia", "godfather"]:
                mafia.append((player["tablePosition"], player["username"]))
            elif player["role"]["type"] == "sheriff":
                sheriff = (player["tablePosition"], player["username"])
                winner = "Мафии" if player["w_l"] == 'lose' else "Города"

        mafia_ids = " ".join([str(x[0]) for x in mafia])
        mafia_usernames = ", ".join([x[1] for x in mafia])

        return f"""
        Мафия: {mafia_ids} ({mafia_usernames});
        LUPA: {sheriff[0]} ({sheriff[1]});
        Победа {winner}"""
    except:
        return None

In [ ]:
#Содержимое бота
class TwitchBot(Bot):
    def __init__(self):
        self.current: str = None
        self.previous: str = None
        self.last_result: str = None
            
        self.op_users: list = [] 
        
        
        super().__init__(login = "kot_when_v_bar",
                         token="vbuvqm4bo0z0zq5vy6z12vwxo8wbu5",
                         refresh_token="0xf86s515ydl9877iga1t50p2294b0ktki6l241msgr6imxo4v",
                         prefix="!",
                         initial_channels=['nekotovsky_'])
        
    async def event_ready(self):
        print(f'Logged in as | {self.nick}')

    async def event_message(self, message):
        if message.echo:
            return

        print(message.author, ": ", message.content)

        await self.handle_commands(message)
        
    @command(aliases=["id"])
    async def remember_current(self, ctx, message: str):
        #TODO сравнить пользователя с op_users
        
        if ctx.author.is_mod or ctx.author.name == 'ii_silentwolf_ii':
            
            if message.isnumeric() and len(message) == 5:

                self.current = message

                #self.last_games[self.previous] = self.last_result
                #self.last_result = None

                status = f"/me Понял SeemsGood {self.current}"
            else:
                status = "/me Не понял NotLikeThis"

            await ctx.send(status)
        else:
            await ctx.send("/me обновлять id могут только модераторы NotLikeThis")
        
    @command()
    async def lastold(self, ctx):
        status = get_players(self.current)
        if status is None:
            await ctx.send(self.last_result)
        else: 
            self.last_result = status
            await ctx.send(status)
    
    @command(aliases=["ласт","дфые"])
    async def last(self, ctx):
        self.current = get_game_id()
        status = get_players(self.current)
        await ctx.send(f'/me {status}')
        
#     async def last(self, ctx):
#         self.current = get_game_id()
#         if self.current == self.previous:
#             await ctx.send(self.last_result)
#         else:
#             status = get_players(self.current)
#             self.previous = self.current
#             self.last_result = status
#             await ctx.send(status)



    @command(aliases=["когда"])
    async def when(self, ctx):
        randomize = ['a', 'b', 'c', 'd', 'e', 'f']
        outcome = choice(randomize)
        if outcome == 'a':
            await ctx.send("/me @nekotovsky_ когда в бар")
        elif outcome == 'b':
            await ctx.send("/me @nekotovsky_ когда Гран Торино")
        elif outcome == 'c':
            await ctx.send("/me @nekotovsky_ когда на шашлыки")
        elif outcome == 'd':
            await ctx.send("/me @nekotovsky_ когда в ссаные картишки")
        elif outcome == 'e':
            await ctx.send("/me @INTMASHINA когда стрим по Геншину")
        elif outcome == 'f':
            await ctx.send("/me @nekotovsky_ когда ASMR-стрим")
            
    @command(aliases=["факт"])
    async def fact(self, ctx):
        randomize = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
        outcome = choice(randomize)
        if outcome == 'a':
            await ctx.send("/me Вот один факт о Некотовском: Он никогда не играл на чёрной карте")
        elif outcome == 'b':
            await ctx.send("/me Вот один факт о Некотовском: Он никогда не ставил руку на 3 в 3")
        elif outcome == 'c':
            await ctx.send("/me Вот один факт о Некотовском: Он никогда не занимал восьмое место в БГ")
        elif outcome == 'd':
            await ctx.send("/me Вот один факт о Некотовском: Он никогда не пробовал алкоголь")
        elif outcome == 'e':
            await ctx.send("/me Вот один факт о Некотовском: Он никогда не врёт")
        elif outcome == 'f':
            await ctx.send("/me Вот один факт о Некотовском: Он кажется старше своего возраста из-за большого роста 135см")
        elif outcome == 'g':
            await ctx.send("/me Вот один факт о Некотовском: Он на сто процентов попадает в цвет Клауза")
    
    @command(aliases=["мод"])
    async def mod(self, ctx):
        if ctx.author.is_mod:
            
            await ctx.send("/me is mod")
        else:
            await ctx.send("/me not mod")
    
    @command(aliases=["хс"])
    async def battletag(self, ctx):
        await ctx.send("Кетовске1337#2253 Осторожно, мой уровень игры примерно на 12к ммр")
        
    
     
    @command(aliases=["клуб"])
    async def club(self, ctx):
        await ctx.send("https://playpolemica.com/clubs/19 🔎 Клуб 🔎 Луп 🔎 ")
        
    @command(aliases=["бот"])
    async def bot(self, ctx):
        randomize = ['a', 'b', 'c', 'd', 'e']
        outcome = choice(randomize)
        if outcome == 'a':
            await ctx.send("/me PowerUpL DarkMode PowerUpR призываю Волка @II_SilentWolf_II PowerUpL DarkMode PowerUpR")
        elif outcome == 'b':
            await ctx.send(f"/me Кто бот? Я бот? А может быть ты бот? @{ctx.author.name} Kappa")
        elif outcome == 'c':
            await ctx.send("/me Я здесь, жалкие кожаные мешки с костями DarkMode")
        elif outcome == 'd':
            await ctx.send("/me На месте SeemsGood")
        elif outcome == 'e':
            await ctx.send("/me @StreamElements Я ботик ты ботик") 
            
#     @command(aliases=["вип"])
#     async def vip(self, ctx):
#         if ctx.author.is_VIP:
            
#             await ctx.send("is vip")
#         else:
#             await ctx.send("not vip")
        
#     @command(aliases=["игра"])
#     async def test(self, ctx, message: str):
        
#         if message.isnumeric() and len(message) == 5:
#             status = get_players(message)
#         else:
#             status = f"Нельзя получить информацию об игре {message}"
        
#         await ctx.send(status)

In [ ]:
bot = TwitchBot()
bot.run()

In [ ]:
await bot.close()

{"rows":
     [
         {"id":"79918",
          "date_start":"2021-09-28 13:18:15",
          "date_ends":"2021-09-28 13:59:20",
          "duration":"0 ч 41 мин",
          "game_mode":"Лига",
          "points":-1.43,
          "sp":{"gained":-1.5,"total":0},
          "role":{"type":"civilian","title":"Мирный"},
          "result":{"title":"Поражение","code":"fail"}
         }
     ],
 "totalCount":"1574"}